# Baseline基线说明

1. 数据加载与处理
2. 模型加载（高层API）
3. 模型预测
4. 保存提交结果

checkpoint未保存

解压数据集

In [2]:
#!unzip -oq /home/aistudio/data/data85133/常规赛：PALM病理性近视预测.zip

# 1. 导入相应的包

In [2]:
import pandas as pd    # 处理xlsx文件
import os              # 文件操作
import time            # 时间记录
from tqdm import tqdm  # 进度条
import cv2 as cv       # 图像处理
import numpy as np     # 数据计算包

import paddle                             
from paddle import nn                        # 网络层API
from paddle import optimizer                 # 优化器API
from paddle import regularizer               # 正则化API
from paddle import metric                    # 评价指标API
from paddle.nn import loss                   # 损失函数API
from paddle.nn import Layer                  # 网络层基类

from paddle.io import Dataset, DataLoader    # 数据加载基类——Dataset，DataLoader——数据加载器
from paddle.vision import transforms         # 图像预处理API

# 2. 导入数据xlsx文件，拼接完整的图片路径

* 训练数据集有xlsx文件需要读取
* 测试数据要构建DataFrame表格存储文件路径信息和标签，方便后边预测数据读取与提交格式

In [3]:
# 训练数据
Image_path = '常规赛：PALM病理性近视预测/Train/fundus_image'                             # 数据存放根目录
Train_data = pd.read_excel('常规赛：PALM病理性近视预测/Train/Classification.xlsx')       # 数据xlsx文件

for i in range(len(Train_data)):                                                       # 将DataFrame表格中的图片补足路径
    Train_data.iloc[i, 0] = os.path.join(Image_path, Train_data.iloc[i, 0])            # 拼接路径

Train_data = Train_data.sample(frac=1.0, random_state=2021).reset_index(drop=True)     # frac=1.0对应随机采样全部样本（表格数据），对应打乱表格
Train_data.head()

,imgName,Label
0,常规赛：PALM病理性近视预测/Train/fundus_image/V0327.jpg,0
1,常规赛：PALM病理性近视预测/Train/fundus_image/V0189.jpg,1
2,常规赛：PALM病理性近视预测/Train/fundus_image/V0176.jpg,1
3,常规赛：PALM病理性近视预测/Train/fundus_image/N0116.jpg,0
4,常规赛：PALM病理性近视预测/Train/fundus_image/H0010.jpg,0


类似的，处理需要预测的数据，不过这时没有给定的xlsx文件，我们需要创建自定义的DataFrame表格

In [4]:
Test_data = []                                                                      # 测试图片路径(数据)
Test_path = '常规赛：PALM病理性近视预测/PALM-Testing400-Images'                      # 测试图片根目录
for _, _, files in os.walk(Test_path):                                              # 获取目录下的所有图片文件
    for i in files:                  # 遍历文件
        Test_data.append([i, 0])     # 添加当前图片文件+一个默认标签0——以对应img,label的格式，方便预测结果存储
Test_data = np.asarray(Test_data)    # 转换datatype
Test_data = pd.DataFrame(Test_data)  # 转换为DataFrame表格数据
Test_data = Test_data.sort_values(by=0, ascending=True).reset_index(drop=True)      # 对读取的文件排序--文件名字支持排序
for i in range(len(Test_data)):                                                     # 拼接完整图片路径
    Test_data.iloc[i, 0] = os.path.join(Test_path, Test_data.iloc[i, 0])
Test_data.head()

,0,1
0,常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0001.jpg,0
1,常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0002.jpg,0
2,常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0003.jpg,0
3,常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0004.jpg,0
4,常规赛：PALM病理性近视预测/PALM-Testing400-Images/T0005.jpg,0


# 3. 构建数据集Dataset自定义类

* 注意传入的数据为DataFrame的表格数据，因此需要定制方法读取表格数据

In [6]:
class Train_Dataset(Dataset):
    '''加载训练集，把数据加载函数拼进来'''
    def __init__(self, df, trans=None):
        super(Train_Dataset, self).__init__()
        self.df = df
        if trans is None:
            self.trans = transforms.Compose([
                transforms.Resize(size=(960, 960)),
                transforms.ToTensor(),
                transforms.Normalize()
            ])
        else:
            self.trans = trans
        self.lens = len(df)

    def __getitem__(self, indexs):
        im_data, im_label = self._load_img_and_label(self.df, indexs)
        im_data = self.trans(im_data)
        return im_data, paddle.to_tensor(im_label)

    def _load_img_and_label(self, df, index):
        '''加载DF中的路径为图片和标签
            df: 输入DF
            index: 第几条数据
        '''
        assert index < self.lens, \
            'please check the index, which has more than the dataset length!'
        im_data = cv.imread(df.iloc[index, 0], cv.COLOR_BGR2RGB)  # 转为RGB数据
        im_label = int(df.iloc[index, 1])  # 标签
        return np.asarray(im_data).astype('float32'), im_label

    def __len__(self):
        return self.lens

class Test_Dataset(Dataset):
    '''加载测试集
        把数据加载函数拼进来
    '''
    def __init__(self, df, trans=None):
        super(Test_Dataset, self).__init__()
        self.df = df
        if trans is None:
            self.trans = transforms.Compose([
                transforms.Resize(size=(960, 960)),  # 保证迁移前后输入特征大小一致
                transforms.ToTensor(),
                transforms.Normalize()
            ])
        else:
            self.trans = trans
        self.lens = len(df)

    def __getitem__(self, indexs):
        im_data, im_label = self._load_img_and_label(self.df, indexs)
        im_data = self.trans(im_data)
        return im_data, paddle.to_tensor(im_label)

    def _load_img_and_label(self, df, index):
        '''加载DF中的路径为图片和标签
            df: 输入DF
            index: 第几条数据
        '''
        assert index < self.lens, \
            'please check the index, which has more than the dataset length!'
        im_data = cv.imread(df.iloc[index, 0], cv.COLOR_BGR2RGB)  # 转为RGB数据
        im_label = int(df.iloc[index, 1])  # 标签
        return np.asarray(im_data).astype('float32'), im_label
    
    def __len__(self):
        return self.lens

# 模型基本参数配置

* 主要是学习率、轮次、类别、批大小、数据集划分大小

In [7]:
# 训练参数-=dict
Train_Paramdict = {
    'data_length':len(Train_data),  # 数据长度
    'train_frac':0.8,              # 训练集比例，原始：0.8
    'num_class':2,                  # 类别，原始：2
    'epoches':50,                   # 训练轮次，原始：5
    'batchsize':8,                 # 批量大小，原始：8
    'lr':0.001,                      # 学习率,原始：0.005
}

# 划分验证集

* 根据默认参数划分

In [8]:
# 数据集划分
Fit_data  = Train_data.iloc[:int(Train_Paramdict['data_length']*Train_Paramdict['train_frac'])]
Eval_data = Train_data.iloc[int(Train_Paramdict['data_length']*Train_Paramdict['train_frac']):]

# 加载数据集

* 将Dataframe传入自定义的Dataset类
* 创建DataLoader加载器

In [9]:
# 数据加载
Fit_dataset = Train_Dataset(Fit_data)
Eval_dataset = Test_Dataset(Eval_data)

Fit_dataloader = DataLoader(Fit_dataset, batch_size=Train_Paramdict['batchsize'], shuffle=True)
Eval_dataloader = DataLoader(Eval_dataset, batch_size=Train_Paramdict['batchsize'])

# 创建模型

* 利用基模型构建基线--MobileNetV1

* 在V1的基础上，选择MolileNetV2进行训练

* 选择较为稳定的优化器、损失函数、评价指标

In [10]:
#创建模型
#model = paddle.vision.models.ResNet(num_classes=2,block=2,depth=100)
model = paddle.vision.models.MobileNetV2(num_classes=2)# 使用paddle自带的基础模型进行基线测试
model = paddle.Model(model)                             # 使用高层API简化训练过程


# 优化器
O = optimizer.Adam(learning_rate=Train_Paramdict['lr'], parameters=model.parameters() )
# 损失函数
L = loss.CrossEntropyLoss()
# 评估指标--这里baseline选用精确率
M = metric.Accuracy()

# 预载模型训练配置
model.prepare(O, L, M)

开始训练

In [13]:

model.fit(
    Fit_dataloader,
    Eval_dataloader,
    epochs = Train_Paramdict['epoches'],
    eval_freq=1,          # 验证频率--几个轮次验证一次
    log_freq=2,          # 日志频率--几个step输出一次训练日志信息
    # save_dir=None,        # 如果需要保存模型，None改成路径
    # save_freq=1,          # 保存频率--几个epoch保存一次
)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/pyt

# 加载预测数据集

In [11]:
# 数据加载
Test_dataset = Test_Dataset(Test_data)
Test_dataloader = DataLoader(Test_dataset, batch_size=Train_Paramdict['batchsize']) 

# 进行模型预测

* 预测的结果是一个多维概率数据

In [15]:
results = model.predict(Test_dataloader)         # 预测结果--概率

Predict begin...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step  2/50 [>.............................] - ETA: 7:24 - 9s/step

# 预测结果处理，提交结果

* 提取提交有效数据
* 修改Test的DataFrame表格适应提交要求
* 注意保存的表头名称以及数据格式

In [16]:
results = np.asarray(results)   

submit_result = []
for i in results[0]:            # 提取结果数据
    i = paddle.to_tensor(i)     # 便于使用paddle的方法
    i = paddle.nn.functional.softmax(i)            # softmax获取预测概率结果
    result = i[:, 1]            # 获取1类别对应的概率--是否病理性
    submit_result += result.numpy().tolist()       # 拼接list结果
submit_result = np.asarray(submit_result) 

In [17]:
Test_data.iloc[:, 1] = submit_result       # 将结果数据用于修改最初的Test数据DataFrame表格中的Label项数据
Submit_data = Test_data.copy()             # 拷贝一份测试数据
Submit_data.columns = ['FileName', 'PM Risk']  # 修改表格表头，以适应提交需要
for i in range(len(Submit_data)):                         # 取出原Test中的图片文件名称--不要根目录
    Submit_data.iloc[i, 0] = Submit_data.iloc[i, 0][-9:]
Submit_data.head()

,FileName,PM Risk
0,T0001.jpg,0.976404
1,T0002.jpg,0.000336
2,T0003.jpg,0.000055
3,T0004.jpg,0.610417
4,T0005.jpg,0.993214


不需要序列号以及数值格式为保留一位浮点数的格式

In [18]:
Submit_data.to_csv('Classification_Results.csv', index=False, float_format="%.1f")       # 保存结果csv